<font color='darkred'> Unless otherwise noted, **this notebook will not be reviewed or autograded.**</font> You are welcome to use it for scratchwork, but **only the files listed in the exercises will be checked.**

---

# Background

Before we move into the exercises, we'll cover some background information to get you started.

In [2]:
import numpy as np
import pandas as pd
import requests
import re

## Markov Chain Simple Example

Markov chains are a way of representing how systems change over time. The main concept behind Markov chains are that they are memoryless, meaning that the next state of a process only depends on the previous state.

![image](https://upload.wikimedia.org/wikipedia/commons/7/7a/Markov_Chain_weather_model_matrix_as_a_graph.png)

The way to read the Markov chain above from [Wikipedia](https://commons.wikimedia.org/w/index.php?curid=25300524) is:
* If I am currently in the sunny state, there is a 10% chance I will go to the rainy state and a 90% chance I will remain in the sunny state
* If I am currently in the rainy state, there is an 50% chance I will go to the sunny state and a 50% chance I will remain in the rainy state

## Transition Matrices

This is what our **transition matrix** will look like for the Markov chain diagram above. Take a minute to interpret the rows and columns of this matrix.

In [3]:
P = np.asarray([.9, .1, .5, .5]).reshape(2,2)
states = ['sunny', 'rainy']

pd.DataFrame(P, index=states, columns=states)

,sunny,rainy
sunny,0.9,0.1
rainy,0.5,0.5


## Predict Tomorrow's Weather

Let's say it's sunny today, we can represent that as:

`today = [1, 0]`

**Predict tomorrow's weather using what you know about today and the transition matrix.**

In [4]:
today = [1, 0]

tomorrow = np.dot(today, P)
tomorrow

array([0.9, 0.1])

In this example, there is a 90% chance it will remain sunny tomorrow, and a 10% chance it'll be rainy.

**Predict the day after tomorrow's weather.**

In [5]:
# Method 1: Multiply tomorrow's weather by the transition matrix
day_after = np.dot(tomorrow, P)
day_after

array([0.86, 0.14])

In [6]:
# Method 2: Multiply today's weather by the transition matrix^2
day_after = np.dot(today, np.linalg.matrix_power(P, 2))
day_after

array([0.86, 0.14])

## Text Generation

Markov chains can also be used for very basic text generation. **Think about every word in a corpus as a state.** We can make a simple assumption that the next word is only dependent on the previous word - which is the basic assumption of a Markov chain. In this exercise, you'll create a text generator which uses only this concept.

## Read in some text to imitate

We are going to generate some text in the style of inspirational quotes, so let's first read in the data.

In [3]:
url = 'https://raw.githubusercontent.com/leontoddjohnson/datasets/main/text/inspiration_quotes.txt'

content = requests.get(url)
quotes_raw = content.text

print(quotes_raw[:1000])

“Healing comes from taking responsibility: to realize that it is you - and no one else - that creates your thoughts, your feelings, and your actions.” —Peter Shepherd

“Life is a journey and if you fall in love with the journey you will be in love forever.” —Peter Hagerty

“When you return to your old hometown, you find it wasn’t the town you missed, but your childhood.” —Earl Wilson

“As we grow old, the beauty steals inward.” —Ralph Waldo Emerson

“Life begins as a quest of the child for the man, and ends as a journey by the man to rediscover the child.” —Sam Ewing

Happiness
“Ultimately your greatest teacher is to live with an open heart.” —Emmanuel (Pat Rodegast)

“Doing what you like is freedom. Liking what you do is happiness.” —Frank Tyger

“We forge the chains we wear in life.” —Charles Dickens

happiness quote
“If you look to others for fulfillment, you will never be fulfilled. If your happiness depends on money, you will never be happy with yourself. Be content with what you 

## Clean up the text data

There are many ways to clean up data before building a text generator. In this case, we'll try to at least just extract the quotes themselves.

*After you complete the exercises, feel free to adjust this section of the process ...*

In [4]:
quotes = quotes_raw.replace('\n', ' ')
quotes = re.split("[“”]", quotes)   # split on the unique “ characters
quotes[:3]

['',
 'Healing comes from taking responsibility: to realize that it is you - and no one else - that creates your thoughts, your feelings, and your actions.',
 ' —Peter Shepherd  ']

In [5]:
# skip the first one, and capture every other element
quotes = quotes[1::2]
quotes[:3]

['Healing comes from taking responsibility: to realize that it is you - and no one else - that creates your thoughts, your feelings, and your actions.',
 'Life is a journey and if you fall in love with the journey you will be in love forever.',
 'When you return to your old hometown, you find it wasn’t the town you missed, but your childhood.']

In [6]:
# create one long corpus of text
corpus = ' '.join(quotes)

# remove long whitespaces (see regex101.com)
corpus = re.sub(r"\s+", " ", corpus)

# remove leading/trailing whitespaces
corpus = corpus.strip()

corpus[:200]

'Healing comes from taking responsibility: to realize that it is you - and no one else - that creates your thoughts, your feelings, and your actions. Life is a journey and if you fall in love with the '

In general, this version of `corpus` should work just fine!

# Exercises

In these exercises, you'll build a `MarkovText` object that takes in a `corpus` of text, and has the capability to generate text using the Markov Property.

In [7]:
from apputil import *

## Exercise 1: Build a Transition Dictionary

Update the `get_term_dict` method to build a term dictionary of Markov states with the following traits:

* The keys should be (unique) tokens in the corpus
* For each key, the value should be a `list` of all the tokens that follow that key
    - E.g., if my total corpus is "Astrid is very kind, is she not?", then my dictionary should include `{... "is": ["very", "she"] ...}`.
    - Decide whether or not to include duplicates (i.e., *every* iteration) in these lists. Then, explain why or why not.

*Hint: You'll likely want to use [`defaultdict(list)`](https://realpython.com/python-defaultdict/#understanding-the-python-defaultdict-type) here.*

Apply the function to the quotes above. Your final output should look something like this:
    
```python
{
    'Healing': ['comes'],
    'comes': ['from', 'the', 'the', ...],
    'from': ['taking', 'aesthetic', 'a'],
    ...
}
```

In [8]:
# Test the implementation
text_gen = MarkovText(corpus)

# First, let's build and examine the term dictionary
term_dict = text_gen.get_term_dict()

# Show some sample entries
print("Sample entries from term dictionary:")
sample_keys = list(term_dict.keys())[:5]
for key in sample_keys:
    print(f"'{key}': {term_dict[key][:5]}...")  # Show first 5 following words

print(f"\nTotal unique words in dictionary: {len(term_dict)}")
print(f"Dictionary built successfully: {text_gen.term_dict is not None}")

Sample entries from term dictionary:
'Healing': ['comes']...
'comes': ['from', 'from']...
'from': ['taking', 'aesthetic', 'a', 'having', 'not']...
'taking': ['responsibility:', 'it.']...
'responsibility:': ['to']...

Total unique words in dictionary: 994
Dictionary built successfully: True


## Exercise 2: Create a text generator

Update the `generate()` method to generate sentences using the Markov property.

- This should use the `.term_dict`, and it should take in the number of terms you want generated, `term_count`.
- Your function should also be able to accept an *optional* user-defined `seed_term` that starts the generator. By default, you can have the function select a random first token.
  - If the user-defined seed term is not in the corpus, raise a `ValueError`.
- Each "next word" should be chosen *at random* given the "current" word and the term dictionary.
  - Consider [`numpy.random.choice`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) for this.

*Hint: think about what happens if the last term in the corpus occurs only **once**.*

In [9]:
text_gen.generate()

'away and watch your wagon to the world’s a game, play it. Life is the'

In [10]:
# Test more comprehensive generation
print("=== Testing Exercise 2: Text Generation ===")
print()

# Test 1: Random start, default length
print("1. Random start (15 words):")
print(text_gen.generate())
print()

# Test 2: Custom seed word
print("2. Starting with 'life' (20 words):")
try:
    result = text_gen.generate(seed_term="life", term_count=20)
    print(result)
except ValueError as e:
    print(f"Error: {e}")
print()

# Test 3: Different seed word
print("3. Starting with 'happiness' (12 words):")
try:
    result = text_gen.generate(seed_term="happiness", term_count=12)
    print(result)
except ValueError as e:
    print(f"Error: {e}")
print()

# Test 4: Error handling - invalid seed
print("4. Testing error handling with invalid seed:")
try:
    result = text_gen.generate(seed_term="invalidword")
    print(result)
except ValueError as e:
    print(f"✓ Correctly caught error: {e}")
print()

# Test 5: Short generation
print("5. Short generation (5 words):")
print(text_gen.generate(term_count=5))

=== Testing Exercise 2: Text Generation ===

1. Random start (15 words):
peace for your childhood. As long at the town you joy. The Nobel. Oscars. The

2. Starting with 'life' (20 words):
life by creditors or tidy a station you will be loved - it is taking responsibility: to your dysfunction. Life

3. Starting with 'happiness' (12 words):
happiness that we go, or loving. The need to prove their gifts.

4. Testing error handling with invalid seed:
✓ Correctly caught error: Seed term 'invalidword' not found in corpus

5. Short generation (5 words):
‘out there,’ stop yourself. Be


*Note: this exercise illustrates both a Markov Chain (with constant transition probabilities) **and** a [Monte Carlo Simulation](https://en.wikipedia.org/wiki/Monte_Carlo_method) (iterative sampling from a constantly defined probability distribution of words)!*

---

# Bonus Exercise

*<font color='darkorange'>The following exercise is completely optional</font>, and will not be reviewed unless explicitly requested. That said, these are interesting to work on if you have the time!*

Adjust your text generator such that we can increase the "state window size" from one word to `k` words. That is, the term dictionary in this version with `k = 2` would look something like this (*note the tuple keys*):

```python
{
    ("Healing", "comes"): ["from", ...],
    ...,
    ("it", "is"): ["you", "very"],
    ...
}
```

Generate some text for `k` values of 1, 2, and 3, and compare the differences in text coherence. Try larger corpuses ...

In [13]:
# Enhanced MarkovText class with k-word state windows
class EnhancedMarkovText(object):
    
    def __init__(self, corpus, k=1):
        """
        Initialize the enhanced Markov text generator.
        
        Parameters:
        - corpus: text corpus for training
        - k: state window size (number of words to use as state)
        """
        self.corpus = corpus
        self.k = k
        self.term_dict = None
    
    def get_term_dict(self):
        """
        Build a transition dictionary with k-word states.
        
        For k=1: {"word": ["next1", "next2", ...]}
        For k=2: {("word1", "word2"): ["next1", "next2", ...]}
        For k=3: {("word1", "word2", "word3"): ["next1", "next2", ...]}
        """
        from collections import defaultdict
        
        term_dict = defaultdict(list)
        tokens = self.corpus.split()
        
        # Build k-word states
        for i in range(len(tokens) - self.k):
            if self.k == 1:
                # Single word state (original implementation)
                current_state = tokens[i]
            else:
                # Multi-word state (tuple)
                current_state = tuple(tokens[i:i+self.k])
            
            next_word = tokens[i + self.k]
            term_dict[current_state].append(next_word)
        
        self.term_dict = dict(term_dict)
        return self.term_dict
    
    def generate(self, seed_term=None, term_count=15):
        """
        Generate text using k-word Markov states.
        
        Parameters:
        - seed_term: starting word(s). Can be a string (for k=1) or tuple (for k>1)
        - term_count: number of words to generate
        """
        import numpy as np
        import random
        
        if self.term_dict is None:
            self.get_term_dict()
        
        # Handle seed term based on k value
        if seed_term is None:
            # Choose random starting state - use random.choice for complex objects
            current_state = random.choice(list(self.term_dict.keys()))
        else:
            # Validate and set seed term
            if self.k == 1:
                if seed_term not in self.term_dict:
                    raise ValueError(f"Seed term '{seed_term}' not found in corpus")
                current_state = seed_term
            else:
                # For k>1, seed_term should be a tuple or we'll try to find a matching state
                if isinstance(seed_term, str):
                    # Find a state that starts with this word
                    matching_states = [state for state in self.term_dict.keys() 
                                     if state[0] == seed_term]
                    if not matching_states:
                        raise ValueError(f"No states found starting with '{seed_term}'")
                    current_state = random.choice(matching_states)
                else:
                    if seed_term not in self.term_dict:
                        raise ValueError(f"Seed state '{seed_term}' not found in corpus")
                    current_state = seed_term
        
        # Initialize generated words
        if self.k == 1:
            generated_words = [current_state]
        else:
            generated_words = list(current_state)
        
        # Generate subsequent words
        for _ in range(term_count - self.k):
            if current_state not in self.term_dict or len(self.term_dict[current_state]) == 0:
                break
            
            # Choose next word - use numpy for lists
            next_word = np.random.choice(self.term_dict[current_state])
            generated_words.append(next_word)
            
            # Update current state for next iteration
            if self.k == 1:
                current_state = next_word
            else:
                # Slide the window: remove first word, add new word
                current_state = current_state[1:] + (next_word,)
        
        return ' '.join(generated_words)

In [14]:
# Test the enhanced Markov text generator with different k values
print("=== Bonus Exercise: Comparing k-word State Windows ===")
print()

# Test with k=1, 2, and 3
k_values = [1, 2, 3]
term_count = 25

for k in k_values:
    print(f"🔹 k={k} (using {k}-word state{'s' if k > 1 else ''}):")
    
    # Create generator with k-word states
    enhanced_gen = EnhancedMarkovText(corpus, k=k)
    term_dict = enhanced_gen.get_term_dict()
    
    print(f"   Dictionary size: {len(term_dict)} states")
    
    # Show sample dictionary entries
    sample_keys = list(term_dict.keys())[:3]
    print("   Sample states:")
    for key in sample_keys:
        if k == 1:
            print(f"     '{key}' → {term_dict[key][:3]}...")
        else:
            print(f"     {key} → {term_dict[key][:3]}...")
    
    # Generate text
    print(f"   Generated text ({term_count} words):")
    generated_text = enhanced_gen.generate(term_count=term_count)
    print(f"     {generated_text}")
    
    print()

print("🔍 Observations:")
print("• k=1: More random, less coherent (each word depends on only 1 previous word)")
print("• k=2: More coherent phrases (each word depends on 2 previous words)")  
print("• k=3: Most coherent but potentially more repetitive (depends on 3 previous words)")
print("• Higher k values require larger corpuses to avoid dead ends")

=== Bonus Exercise: Comparing k-word State Windows ===

🔹 k=1 (using 1-word state):
   Dictionary size: 994 states
   Sample states:
     'Healing' → ['comes']...
     'comes' → ['from', 'from']...
     'from' → ['taking', 'aesthetic', 'a']...
   Generated text (25 words):
     expression and easy way. Standing for they will simply what you’re doing is not have peace with that most frightens us. It turns what is

🔹 k=2 (using 2-word states):
   Dictionary size: 2150 states
   Sample states:
     ('Healing', 'comes') → ['from']...
     ('comes', 'from') → ['taking', 'not']...
     ('from', 'taking') → ['responsibility:']...
   Generated text (25 words):
     doing is in everyone. And as we let our light shine, we unconsciously give other people around you won’t feel insecure. We are all meant

🔹 k=3 (using 3-word states):
   Dictionary size: 2454 states
   Sample states:
     ('Healing', 'comes', 'from') → ['taking']...
     ('comes', 'from', 'taking') → ['responsibility:']...
     ('fr

In [15]:
# Compare text coherence with the same starting word
print("=== Coherence Comparison: All Starting with 'life' ===")
print()

seed_word = "life"
for k in [1, 2, 3]:
    print(f"k={k}: ", end="")
    enhanced_gen = EnhancedMarkovText(corpus, k=k)
    enhanced_gen.get_term_dict()
    
    try:
        result = enhanced_gen.generate(seed_term=seed_word, term_count=20)
        print(result)
    except ValueError as e:
        print(f"Error: {e}")
    print()

print("📊 Analysis:")
print("• k=1: Chaotic jumps between topics")
print("• k=2: Better flow and more natural phrases")  
print("• k=3: Most coherent and grammatically correct sentences")
print("• Trade-off: Higher k = more coherent but potentially more repetitive")

=== Coherence Comparison: All Starting with 'life' ===

k=1: life is life, fight for they shall become YOUR DESTINY. Wisdom is nothing to share of us grateful, but our

k=2: life to be true. I am bound to live in the way things are. When you realize you contain what

k=3: life by what we get, we make a life by what we give. Money doesn’t bring happiness and creativity. Your

📊 Analysis:
• k=1: Chaotic jumps between topics
• k=2: Better flow and more natural phrases
• k=3: Most coherent and grammatically correct sentences
• Trade-off: Higher k = more coherent but potentially more repetitive


## 🎉 Bonus Exercise Complete!

### Key Achievements:
1. **Extended the Markov model** from single-word states to k-word states
2. **Implemented tuple-based state transitions** for multi-word contexts
3. **Demonstrated improved text coherence** with higher k values
4. **Showed the trade-offs** between coherence and diversity

### Technical Implementation:
- **k=1**: Traditional single-word Markov chains `"word" → ["next1", "next2", ...]`
- **k=2**: Two-word context states `("word1", "word2") → ["next1", "next2", ...]` 
- **k=3**: Three-word context states for maximum coherence

### Results Summary:
- **Higher k values produce more coherent text** but require larger training corpora
- **k=3 generates the most natural-sounding sentences** in our tests
- **The sliding window approach** maintains context throughout generation
- **Error handling** gracefully manages missing states and invalid seeds

This demonstrates how increasing the "memory" of a Markov chain dramatically improves the quality of generated text while maintaining the probabilistic nature of the model!